<a href="https://colab.research.google.com/github/hmghaly/km/blob/main/html_processing_templating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')
cwd="/content/drive/MyDrive/KM"
os.chdir(cwd)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!unzip ui5.zip

Archive:  ui5.zip
  inflating: about-content.txt       
  inflating: category.html           
  inflating: country-selection.html  
  inflating: dashboard-template.html  
  inflating: discalimer-content.txt  
  inflating: index.html              
  inflating: privacy.html            
  inflating: privacy-text.html       
  inflating: tc-content.txt          
  inflating: template.html           
  inflating: terms.html              
replace test.html? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
   creating: assets/
   creating: assets/css/
  inflating: assets/css/animate.min.css  
  inflating: assets/css/bootstrap.min.css  
  inflating: assets/css/default.css  
  inflating: assets/css/font-awesome.min.css  
  inflating: assets/css/form.css     
  inflating: assets/css/multi-step-form.css  
  inflating: assets/css/style.css    
   creating: assets/fonts/
  inflating: assets/fonts/fa-brands-400.eot  
  inflating: assets/fonts/fa-brands-400.html  
  inflating: assets/fonts/fa-brands-400.ttf  

In [ ]:
from google.colab.output import eval_js
cur_address=eval_js("google.colab.kernel.proxyPort(8000)")
ui_address=cur_address+"ui5"
print(ui_address)
#https://jb5m2hwwxhn-496ff2e9c6d22116-8000-colab.googleusercontent.com/ui5/


https://jb5m2hwwxhn-496ff2e9c6d22116-8000-colab.googleusercontent.com/ui5


In [ ]:
!python -m http.server 8000

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [06/Apr/2022 20:06:00] "GET /ui5 HTTP/1.1" 301 -
127.0.0.1 - - [06/Apr/2022 20:06:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 20:06:35] code 404, message File not found
127.0.0.1 - - [06/Apr/2022 20:06:35] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2022 20:06:47] "GET /ui5/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 20:06:47] "GET /ui5/assets/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 20:06:47] code 404, message File not found
127.0.0.1 - - [06/Apr/2022 20:06:47] "GET /b2web_ui/js/script.js HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2022 20:06:47] "GET /ui5/assets/css/form.css HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 20:06:47] "GET /ui5/assets/css/multi-step-form.css HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 20:06:47] "GET /ui5/assets/css/font-awesome.min.css HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2022 20:06:47] "GET /ui5/assets/css/default.css HTTP/1.1" 200 -
127.0.0.1 

In [10]:
import re

def get_attrs(tag0):
  found_attrs=re.findall('(\S+)="(.+?)"',tag0)
  return dict(iter(found_attrs))

class element:
  def __init__(self) -> None:
      self.name=""
      self.id=""
      self.assigned_id=""
      self.className=""
      self.class_list=[]
      self.tag_name=""
      self.attrs={}
      self.parent=None #must have only one parent
      self.children=[] #only direct children - not recursive
      self.tag_type=None #comment/stand-alone/text-node
      self.open_tag=""
      self.close_tag=""
      self.text="" #if it is a text node
      self.open_tag_i0=None
      self.open_tag_i1=None
      self.close_tag_i0=None
      self.close_tag_i1=None
      self.inner_html=""
      self.outer_html=""
      self.href=""
      self.src=""

class DOM:
  def __init__(self,content) -> None:
    self.content=content
    self.tag_dict={} #linking assigned IDs with element objects
    self.id_dict={} #mapping actual IDs with assigned IDs
    self.actual_ids=[] #a list of IDs actually used, can help us spot duplicate IDs
    self.class_id_dict={} #map each class name to the assigned ids of elements with that class
    self.text_items=[]
    self.tag_id_list=[] #list of assigned IDs
    self.all_links=[]
    self.all_images=[]
    self.description=""
    self.title=""
    tags=list(re.finditer('<([^<>]*?)>', self.content))
    open_tags=[""]
    tag_counter_dict={}
    start_i=0
    for ti_, t in enumerate(tags):
      tag_str,tag_start,tag_end=t.group(0), t.start(), t.end()
      inter_text=self.content[start_i:tag_start] #intervening text since last tag
      last_open_tag=open_tags[-1]
      if len(inter_text)>0:
        if not last_open_tag in ["script","style"]: self.text_items.append(inter_text)
        text_node_count=tag_counter_dict.get("text_node",0)
        text_node_id="text_node_%s"%text_node_count
        tag_counter_dict["text_node"]=text_node_count+1
        text_el=element()
        text_el.text=inter_text
        text_el.tag_type="text_node"
        text_el.parent=self.tag_dict.get(open_tags[-1])
        self.tag_dict[text_node_id]=text_el
        if text_el.parent!=None: self.tag_dict[open_tags[-1]].children+=[text_node_id]
      start_i=tag_end
      tag_str_lower=tag_str.lower()
      tag_name=re.findall(r'</?(.+?)[\s>]',tag_str_lower)[0]
      if tag_name.startswith("h") or tag_name in ["p","div","br","li"]: self.text_items.append("<br>")
      tag_count=tag_counter_dict.get(tag_name,0)
      assigned_tag_id="%s_%s"%(tag_name,tag_count)
      tag_counter_dict[tag_name]=tag_count+1
      cur_el=element()
      cur_el.assigned_id=assigned_tag_id
      tag_type=""
      if tag_str.startswith('</'): tag_type="closing"
      elif tag_str.startswith('<!'): tag_type="comment"
      elif tag_str_lower.endswith('/>') or tag_name in ["input","link","meta","img","br"]: tag_type="s" #standalone
      else: tag_type="opening"
      if tag_type=="closing" and len(open_tags)>0: #if it is a closing tag,
        if tag_name==open_tags[-1].split("_")[0]: #we check if the tag name matches the last open tag name
          el_to_close=self.tag_dict[open_tags[-1]] #and then identify the element corresponding to the last open tag
          tmp_open_i0,tmp_open_i1=el_to_close.open_tag_i0,el_to_close.open_tag_i1 #retrieve the start and end indexes/locations from the open tag
          tmp_inner_html=self.content[tmp_open_i1:tag_start] #then get inner html
          tmp_outer_html=self.content[tmp_open_i0:tag_end] #then outer html
          el_to_close.inner_html=tmp_inner_html #and assign these to the retrieved element object
          el_to_close.outer_html=tmp_outer_html
          if tag_name=="title": self.title=tmp_inner_html
          if tag_name=="a": self.all_links.append(el_to_close)
          
            
          open_tags=open_tags[:-1]
        else: print("open_tags",open_tags, "tag_name",tag_name)
      else:
        self.tag_id_list.append(assigned_tag_id)
        cur_el=element()
        cur_el.open_tag=tag_str 
        cur_el.open_tag_i0,cur_el.open_tag_i1=tag_start,tag_end
        cur_el.tag_name=tag_name
        
        
        cur_el.assigned_id=assigned_tag_id
        cur_el.tag_type=tag_type
        cur_el.attrs=get_attrs(tag_str)
        cur_id=cur_el.attrs.get("id")
        cur_class_str=cur_el.attrs.get("class","")
        cur_class_list=[v for v in cur_class_str.split(" ") if v]

        if tag_name=="meta" and cur_el.attrs.get("name","")=="description":self.description=cur_el.attrs.get("content","")
        if cur_el.attrs.get("href")!=None: cur_el.href=cur_el.attrs.get("href") #self.all_links.append(cur_el.attrs.get("href"))
        if cur_el.attrs.get("src")!=None: cur_el.src=cur_el.attrs.get("src")
        if tag_name=="img": self.all_images.append(cur_el)

        if cur_id!=None:
          cur_el.id=cur_id
          self.actual_ids.append(cur_id)
          self.id_dict[cur_id]=assigned_tag_id
        if tag_name in ["script","style"]: cur_el.tag_type=tag_name
        cur_el.parent=self.tag_dict.get(open_tags[-1])
        #print("cur_el.parent",cur_el.parent,"assigned_tag_id",assigned_tag_id)
        if cur_el.parent!=None: self.tag_dict[open_tags[-1]].children+=[assigned_tag_id]
        #else: print("Parent not found:", open_tags[-1], "tag info", assigned_tag_id,tag_type,tag_str_lower,"open_tags",open_tags)


        if tag_type=="opening": 
          cur_el.close_tag='</%s>'%tag_name #for regular tags with open-close
          open_tags.append(assigned_tag_id)
        self.tag_dict[assigned_tag_id]=cur_el
        for cls0 in cur_class_list:
          self.class_id_dict[cls0]=self.class_id_dict.get(cls0,[])+[assigned_tag_id]
          
  def get_html(self,assigned_tag_id0,html_content0=''):
    cur_el=self.tag_dict.get(assigned_tag_id0)
    if cur_el==None: return html_content0
    html_content0+=cur_el.open_tag
    html_content0+=cur_el.text
    children=cur_el.children
    for ch0 in children:
      html_content0=self.get_html(ch0,html_content0)
      #ch_obj=self.tag_dict.get(ch0)
      #html_content0+=ch_obj.
    html_content0+=cur_el.close_tag
    return html_content0
  def get_el_by_id(self,actual_id):
    cur_assigned_id=self.id_dict.get(actual_id)
    if cur_assigned_id==None: return None
    return self.tag_dict[cur_assigned_id]
  def get_el_by_class(self,class_name0):
    cur_ids=self.class_id_dict.get(class_name0,[])
    el_list=[]
    for id0 in cur_ids: el_list.append(self.tag_dict[id0])
    return el_list
  def apply_content_by_id(self,id0,el_content0,new_attrs_dict0={}):
    repl_list=[]
    cur_el=self.get_el_by_id(id0)
    if cur_el==None: return repl_list
    new_open_tag=cur_el.open_tag
    if new_attrs_dict0!={}:
      cur_attrs=cur_el.attrs
      for a0,b0 in new_attrs_dict0.items():
        cur_attrs[a0]=b0
      new_open_tag=create_open_tag(cur_el.tag_name,cur_attrs)
    cur_outer_html=cur_el.outer_html
    if el_content0: new_outer_html=new_open_tag+el_content0+cur_el.close_tag
    else: new_outer_html=new_open_tag+cur_el.inner_html+cur_el.close_tag
    # print("cur_outer_html",cur_outer_html)
    # print("new_outer_html",new_outer_html)
    repl_list.append((cur_outer_html,new_outer_html))
    return repl_list
  def apply_content_by_class(self,class0,el_content0,new_attrs_dict0={}):
    repl_list=[]
    cur_el_list=self.get_el_by_class(class0)
    for cur_el in cur_el_list:
      new_open_tag=cur_el.open_tag
      if new_attrs_dict0!={}:
        cur_attrs=cur_el.attrs
        for a0,b0 in new_attrs_dict0.items():
          cur_attrs[a0]=b0
        new_open_tag=create_open_tag(cur_el.tag_name,cur_attrs)      
      cur_outer_html=cur_el.outer_html
      if el_content0: new_outer_html=new_open_tag+el_content0+cur_el.close_tag
      else: new_outer_html=new_open_tag+cur_el.inner_html+cur_el.close_tag      
      repl_list.append((cur_outer_html,new_outer_html))
    return repl_list
  def get_repl_pairs(self,cur_repl_dict0):
    repl_pairs=[]
    for key0,val0 in cur_repl_dict0.items():
      new_content0,new_attrs0=val0 #replacement dict has the values as tuples of new content and new attrs
      if key0.startswith("#"): #we follow jquery selectors, # indicates selection by ID, while . indicates selection by class name
        repl_pairs.extend(self.apply_content_by_id(key0[1:],new_content0,new_attrs0))
      elif key0.startswith("."):
        repl_pairs.extend(self.apply_content_by_class(key0[1:],new_content0,new_attrs0))
    return repl_pairs
  def replace(self,repl_dict0):
    new_content=str(self.content)
    cur_repl_pairs=self.get_repl_pairs(repl_dict0)
    for a,b in cur_repl_pairs:
      new_content=new_content.replace(a,b)
    return new_content
  # def get_title(self):
  #   title_str=""
  #   title_el=self.tag_dict.get("title_0")
  #   if title_el!=None: title_str= title_el.inner_html
  #   return title_str

def create_open_tag(tag_name0,tag_attrs0={},self_closing=False):
  attr_items=[]
  for a,b in tag_attrs0.items():
    attr_items.append('%s="%s"'%(a,b))
  attr_str=" ".join(attr_items)
  if self_closing: final_open_tag='<%s %s/>'%(tag_name0,attr_str)
  else: final_open_tag='<%s %s>'%(tag_name0,attr_str)
  return final_open_tag



cur_fpath="ui5/index.html"
cur_fpath="ui5/category.html"
fopen=open(cur_fpath)
cur_content=fopen.read()
fopen.close()


cur_dom_obj=DOM(cur_content)
actual_ids0=cur_dom_obj.actual_ids
tag_dict0=cur_dom_obj.tag_dict
id_dict0=cur_dom_obj.id_dict

print(len(actual_ids0),actual_ids0[:100])
cur_id='main_directory_div'
cur_id='directory_col1'
cur_id='test123'
#cur_id='header'
#cur_id='side_menu'
cur_el=cur_dom_obj.get_el_by_id(cur_id)
repl_list0=cur_dom_obj.apply_content_by_id(cur_id,"testing 12322424",{"class":"test-class","onclick":"whatever()"})
# for a in repl_list0:
#   print(">>>>", a)

# print("outer:", cur_el.outer_html)
# print("--------")

cur_class_name="fa-dribbble"
cur_class_name="active"
cls0='country'
#cls0='fa-times'
cur_class_id_list=cur_dom_obj.get_el_by_class(cls0) #class_id_dict.get(cur_class_name,[])
# for a in cur_class_id_list:
#   print(a.outer_html)
#   print("------")
#repl_list0=cur_dom_obj.apply_content_by_class(cls0,"Australia",{"class":"anything","onclick":"do_something()"})
# for a in repl_list0:
#   print(a)
#   print("------")
test_repl_dict={}
test_repl_dict["#test123"]=("Now we are testing replacing with IDs",{"class":"new-class id-selection","onclick":"do_it()"})
test_repl_dict[".country"]=("Australia",{"class":"new-class class-selection","onclick":"class_function()"})
repl_list1=cur_dom_obj.get_repl_pairs(test_repl_dict)
for a,b in repl_list1:
  print(a)
  print(b)
  print("-----")

all_text_items="".join(cur_dom_obj.text_items)
all_text_items_split=[v.strip() for v in all_text_items.split("<br>") if v.strip()] 
# for a in all_text_items_split:
#   print([a])
#   print("-----")

title0=cur_dom_obj.title
desc0=cur_dom_obj.description
all_links=cur_dom_obj.all_links
all_images=cur_dom_obj.all_images
# print("title0",title0)
# print("desc0",desc0)

# for a in all_images:
#   print(a.src,[a.inner_html])

for a0 in all_links:
  print(a0.href, [a0.outer_html])
  print("--------")
#print("inner:", cur_el.inner_html)


# corr_id=id_dict0[cur_id]
# test_html=cur_dom_obj.get_html(corr_id)
# print(corr_id)
# print(test_html)
#div_79


open_tags ['', 'html_0', 'body_0', 'divclass="off_canvars_overlay"_0'] tag_name div
open_tags ['', 'html_0', 'body_0', 'divclass="off_canvars_overlay"_0'] tag_name body
open_tags ['', 'html_0', 'body_0', 'divclass="off_canvars_overlay"_0'] tag_name html
65 ['side_menu', 'menu', 'header', 'page_title', 'cat_name', 'cat_description', 'cat_children', 'prev_page', 'results', 'myTab', 'home-tab', 'com_category_count', 'home-tab', 'com_category_count', 'profile-tab', 'gov_category_count', 'contact-tab', 'edu_category_count', 'contact-tab', 'org_category_count', 'contact-tab', 'result_list', 'pagination', 'search', 'input_category_search', 'footer', 'loginModal', 'loginModalLabel', 'registerModal', 'registerModalLabel', 'signup_user_email', 'businessModal', 'businessModalLabel', 'app', 'progressbar', 'name_of_business', 'bussiness_website_email', 'business_desc', 'bussiness_email', 'check', 'bussiness_email', 'bussiness_email', '" class=', 'other', 'r_number', 'terms', 'submit', 'contactModal

In [63]:
test_tag=create_open_tag("a",{"href":"www.bbc.com","target":"new"})
print(test_tag)

<a href="www.bbc.com" target="new">


In [ ]:
children0=tag_dict["body_0"].children
for ch in children0:
  ch_obj=tag_dict[ch]
  print(ch_obj.open_tag)
  print([ch_obj.text])
  print("----")


['  \n\n    ']
----
<!--====== PRELOADER PART START ======-->
['']
----

['\n\n    ']
----
<div class="loader-wrap">
['']
----

['\n\n    ']
----
<!--====== PRELOADER PART ENDS ======-->
['']
----

['\n\n    ']
----
<!--====== OFFCANVAS MENU PART START ======-->
['']
----

['\n\n    ']
----
<div class="off_canvars_overlay">
['']
----

['\n    ']
----
<div id="side_menu" class="offcanvas_menu">
['']
----

['\n\n    ']
----
<!--====== OFFCANVAS MENU PART ENDS ======-->
['']
----

['\n   \n    ']
----
<!--====== PART START ======-->
['']
----

['\n    \n    ']
----
<header id="header" class="app-header-area app-header-page-area app-sticky">
['']
----

['\n    \n    ']
----
<!--====== PART ENDS ======-->
['']
----

['\n\n    ']
----
<!--====== app PAGE TITLE PART START ======-->
['']
----

['\n    \n    ']
----
<div id="page_title" class="app-page-title-area">
['']
----

['\n    \n    ']
----
<!--====== app PAGE TITLE PART ENDS ======-->
['']
----

['\n\n    ']
----
<section id="categorie

In [ ]:
def get_attrs(tag0):
  found_attrs=re.findall('(\S+)="(.+?)"',tag0)
  return dict(iter(found_attrs))

test='<section id="categories" class="app-category-area pb-100 pt-90">'
test='<div class="modal fade" id="businessModal" tabindex="-1" aria-labelledby="businessModalLabel" aria-hidden="true">'
test='<header id="header" class="app-header-area app-header-page-area app-sticky">'
get_attrs(test)

{'class': 'app-header-area app-header-page-area app-sticky', 'id': 'header'}